<h2 id="exercises-i">Exercises I</h2>
<p>Run <code>python -m pip install pymysql</code> from your terminal to install pymysql.</p>
<p>Create a notebook or python script named <code>advanced_dataframes</code> to do your work in for these exercises.</p>


1. Run <code>python -m pip install pymysql</code> from your terminal to install the mysql client (any folder is fine)</li>

2. cd into your exercises folder for this module and run <code>echo env.py &gt;&gt; .gitignore</code></li>

3. <p>Create a function named <code>get_db_url</code>. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.</p>

In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_db_url(db_name):
    from env import user, host, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

4. <p>Use your function to obtain a connection to the <code>employees</code> database.</p>

In [3]:
url = get_db_url('employees')

In [ ]:
employees = pd.read_sql('SELECT * FROM employees', url)
employees

5. <p>Once you have successfully run a query:</p>
<p>a. Intentionally make a typo in the database url. What kind of error message do you see?</p>

<p>b. Intentionally make an error in your SQL query. What does the error message look like?</p>

In [ ]:
pd.read_sql('SELECT * FROM emploees LIMIT 5 OFFSET 50', url)

6. <p>Read the <code>employees</code> and <code>titles</code> tables into two separate DataFrames.</p>

In [ ]:
employees = pd.read_sql('SELECT * FROM employees', url)

In [ ]:
employees

In [ ]:
employees.shape

In [ ]:
def get_db_url(db_name):
    from env import user, host, password
    return f'mysql+pymysql://{user}:{password}@{host}/titles'

In [ ]:
url2 = get_db_url('titles')

7. <p>How many rows and columns do you have in each DataFrame? Is that what you expected?</p>

In [ ]:
query = '''
SELECT
    t.title as title,
    d.dept_name as dept_name,
    t.to_date as to_date,
    t.from_date as from_date
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
'''

title_dept = pd.read_sql(query, url)
title_dept.head()

8. <p>Display the summary statistics for each DataFrame.</p>


In [ ]:
employees.describe()

In [ ]:
title_dept.shape

In [ ]:
employees.shape

In [ ]:
title_dept.describe()

9. <p>How many unique titles are in the <code>titles</code> DataFrame?</p>

In [ ]:
# 7

10. <p>What is the oldest date in the <code>to_date</code> column? </p>

In [ ]:
title_dept.sort_values(by = 'to_date', ascending = True).head(1)

In [ ]:
title_dept[title_dept.to_date == title_dept.to_date.min()]

11. <p>What is the most recent date in the <code>to_date</code> column?</p>

In [ ]:
title_dept.sort_values(by = 'to_date', ascending = False)

In [ ]:
title_dept[title_dept.to_date == title_dept.to_date.max()]

In [ ]:
url = get_db_url("employees")
sql = '''
SELECT * FROM titles where to_date != '9999-01-01'
'''
titles2 = pd.read_sql(sql, url)
titles2.head()

In [ ]:
title_dept[(title_dept.to_date.astype(str) < '9999-01-01')].max()

In [ ]:
titles2.to_date.max()

In [ ]:
titles2.to_date = pd.to_datetime(titles2.to_date)

In [ ]:
titles2.sort_values(by = 'to_date')

# Exercises II

1. <p>Copy the <code>users</code> and <code>roles</code> DataFrames from the examples above. </p>

In [2]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [3]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


2. <p>What is the result of using a <code>right</code> join on the DataFrames? </p>

In [5]:
users_right = users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)
users_right

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


3. <p>What is the result of using an <code>outer</code> join on the DataFrames?</p>
</li>

In [6]:
users_outer = users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)
users_outer

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


4. <p>What happens if you drop the foreign keys from the DataFrames and try to merge them?</p>

5. <p>Load the <code>mpg</code> dataset from PyDataset. </p>


In [7]:
from pydataset import data


In [9]:
mpg = data('mpg') # load the dataset and store it in a variable
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


6. <p>Output and read the documentation for the <code>mpg</code> dataset.</p>

In [10]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




7. <p>How many rows and columns are in the dataset?</p>


In [12]:
mpg.shape

(234, 11)

8. <p>Check out your column names and perform any cleanup you may want on them.</p>

In [13]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [14]:
mpg.rename(columns={'cty': 'city', 'hwy':'highway', 'disp':'displacement', 'cyl':'cylinders'}, inplace = True)

9. <p>Display the summary statistics for the dataset.</p>


In [15]:
mpg.describe()

,displ,year,cylinders,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


10. <p>How many different manufacturers are there?</p>


In [17]:
len(mpg['manufacturer'].unique())

15

11. <p>How many different models are there?</p>


In [18]:
len(mpg['model'].unique())

38

12. <p>Create a column named <code>mileage_difference</code> like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.</p>

In [19]:
mpg['mileage_difference'] = mpg.highway - mpg.city
mpg

,manufacturer,model,displ,year,cylinders,trans,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


13. <p>Create a column named <code>average_mileage</code> like you did in the DataFrames exercises; this is the mean of the city and highway mileage.</p>

In [20]:
mpg['average_mileage'] = round((mpg.highway + mpg.city)/2,0).astype(int)  
mpg

,manufacturer,model,displ,year,cylinders,trans,drv,city,highway,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,24
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,26
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,26
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,24
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22


14. <p>Create a new column on the <code>mpg</code> dataset named <code>is_automatic</code> that holds boolean values denoting whether the car has an automatic transmission.</p>

In [21]:
mpg['is_automatic'] = mpg.trans.str.startswith('a')
mpg

,manufacturer,model,displ,year,cylinders,trans,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,24,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,26,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,26,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,24,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22,False


15.  <p>Using the <code>mpg</code> dataset, find out which which manufacturer has the best miles per gallon on average?</p>

In [29]:
mpg.groupby('manufacturer').average_mileage.agg(['mean']).sort_values(by='mean', ascending = False).head(1)

,mean
manufacturer,
honda,28.333333


16. <p>Do automatic or manual cars have better miles per gallon?</p>


In [32]:
mpg.groupby('is_automatic').average_mileage.agg(['mean']).sort_values(by='mean', ascending = False)

,mean
is_automatic,
False,22.220779
True,19.133758
